In [ ]:
import hashlib

# try a md5 hash and apply it to string
def string2numeric_hash(text):
    return int(hashlib.md5(text.encode('utf-8')).hexdigest()[:8], 16)
 
# test
print (string2numeric_hash("this is a nice string"))
print (string2numeric_hash("loser"))

In [ ]:
import pandas as pd
X_train = pd.read_csv("data/train")

In [ ]:
X_test = pd.read_csv("data/test")

In [ ]:
X_test.head().site_id

In [ ]:
X_train['site_id_md5']=X_train.site_id.apply(string2numeric_hash)
X_train['site_domain_md5']=X_train.site_domain.apply(string2numeric_hash)
X_train['site_category_md5']=X_train.site_category.apply(string2numeric_hash)

X_train['app_id_md5']=X_train.app_id.apply(string2numeric_hash)
X_train['app_domain_md5']=X_train.app_domain.apply(string2numeric_hash)
X_train['app_category_md5']=X_train.app_category.apply(string2numeric_hash)

X_train['device_id_md5']=X_train.device_id.apply(string2numeric_hash)
X_train['device_ip_md5']=X_train.device_ip.apply(string2numeric_hash)
X_train['device_model_md5']=X_train.device_model.apply(string2numeric_hash)

In [ ]:
X_train['weekday']= (X_train['hour']%10000/100).astype(int)%7
X_train['hour_']= (X_train['hour']%100)

In [ ]:
tmp_features = X_train[['site_id_md5','site_domain_md5','site_category_md5',\
                        'app_id_md5','app_domain_md5','app_category_md5',\
                        'device_id_md5','device_ip_md5','device_model_md5',\
                        'weekday','hour_','banner_pos','C1','device_type','device_conn_type',\
                        'C14','C15','C16','C17','C18','C19','C20','C21']].values

In [ ]:
labels = X_train.click.values

In [ ]:
import xgboost as xgb
import numpy as np
param = {'max_depth':8, 'eta':0.1, 'silent':1, 'objective':'binary:logistic' ,'subsample':0.3}
param['nthread'] = 4
param['colsample_bytree'] = 0.5
param['eval_metric'] = 'logloss'

#msk = np.random.rand(len(X_train)) < 0.8
msk = np.load("20170727_msk.npy")

dtrain = xgb.DMatrix(tmp_features[msk], labels[msk])
dvali = xgb.DMatrix(tmp_features[~msk], labels[~msk])

In [ ]:
num_round = 80000
evallist  = [ (dtrain,'train'),(dvali,'eval')]
bst = xgb.train( param, dtrain, num_round, evallist , early_stopping_rounds=500)

In [ ]:
bst.best_score

In [ ]:
bst.save_model('md_hash_0803.model')

# testing phase

In [ ]:
import pandas as pd
X_test = pd.read_csv("data/test")

In [ ]:
import hashlib

# try a md5 hash and apply it to string
def string2numeric_hash(text):
    return int(hashlib.md5(text.encode('utf-8')).hexdigest()[:8], 16)
 
# test
print (string2numeric_hash("this is a nice string"))
print (string2numeric_hash("loser"))

In [ ]:
X_test['site_id_md5'] = X_test.site_id.apply(string2numeric_hash)
X_test['site_domain_md5'] = X_test.site_domain.apply(string2numeric_hash)
X_test['site_category_md5'] = X_test.site_category.apply(string2numeric_hash)

X_test['app_id_md5'] = X_test.app_id.apply(string2numeric_hash)
X_test['app_domain_md5'] = X_test.app_domain.apply(string2numeric_hash)
X_test['app_category_md5'] = X_test.app_category.apply(string2numeric_hash)

X_test['device_id_md5'] = X_test.device_id.apply(string2numeric_hash)
X_test['device_ip_md5'] = X_test.device_ip.apply(string2numeric_hash)
X_test['device_model_md5'] = X_test.device_model.apply(string2numeric_hash)

In [ ]:
X_test['weekday']= (X_test['hour']%10000/100).astype(int)%7
X_test['hour_']= (X_test['hour']%100)

In [ ]:
tmp_features = X_test[['site_id_md5','site_domain_md5','site_category_md5',\
                        'app_id_md5','app_domain_md5','app_category_md5',\
                        'device_id_md5','device_ip_md5','device_model_md5',\
                        'weekday','hour_','banner_pos','C1','device_type','device_conn_type',\
                        'C14','C15','C16','C17','C18','C19','C20','C21']].values

In [ ]:
import xgboost as xgb

In [ ]:
bst = xgb.Booster({'nthread':4}) #init model
bst.load_model("md_hash_0803.model") # load data

In [ ]:
dtest = xgb.DMatrix(tmp_features)
#ypred = bst.predict(dtest)
ypred = bst.predict(dtest,ntree_limit=500)

In [ ]:
sampleSubmission = pd.read_csv("data/sampleSubmission")
#sampleSubmission.click = (y_site+y_app)/2
sampleSubmission.click = ypred

# with a naive feature hashing (0.408, 1142/1604)
sampleSubmission.to_csv("md_hash_0803.csv",index=False)

In [ ]:
dtest = xgb.DMatrix(tmp_features)
#ypred = bst.predict(dtest)
ypred = bst.predict(dtest,ntree_limit=250)
sampleSubmission = pd.read_csv("data/sampleSubmission")
#sampleSubmission.click = (y_site+y_app)/2
sampleSubmission.click = ypred

# with a naive feature hashing (0.408, 1142/1604)
sampleSubmission.to_csv("md_hash_0803_250.csv",index=False)

In [ ]:
dtest = xgb.DMatrix(tmp_features)
#ypred = bst.predict(dtest)
ypred = bst.predict(dtest,ntree_limit=1000)
sampleSubmission = pd.read_csv("data/sampleSubmission")
#sampleSubmission.click = (y_site+y_app)/2
sampleSubmission.click = ypred

# with a naive feature hashing (0.408, 1142/1604)
sampleSubmission.to_csv("md_hash_0803_1000.csv",index=False)

In [ ]:
dtest = xgb.DMatrix(tmp_features)
#ypred = bst.predict(dtest)
ypred = bst.predict(dtest,ntree_limit=1500)
sampleSubmission = pd.read_csv("data/sampleSubmission")
#sampleSubmission.click = (y_site+y_app)/2
sampleSubmission.click = ypred

# with a naive feature hashing (0.408, 1142/1604)
sampleSubmission.to_csv("md_hash_0803_1500.csv",index=False)

# [Sec2] handle not interception training and testing as missing

In [ ]:
import pandas as pd
X_train = pd.read_csv("data/train")

In [ ]:
X_test = pd.read_csv("data/test")

## site_id

In [ ]:
site_id_train = X_train.site_id.value_counts()/len(X_train)
site_id_test = X_test.site_id.value_counts()/len(X_test)
intersection = site_id_test.index.intersection(site_id_train.index)

In [ ]:
def missing_replace(x):
    if x not in intersection:
        return "missing"
    else:
        return x

In [ ]:
X_train.site_id = X_train.site_id.apply(missing_replace)
X_test.site_id = X_test.site_id.apply(missing_replace)

## site_domain

In [ ]:
site_domain_train = X_train.site_domain.value_counts()/len(X_train)
site_domain_test = X_test.site_domain.value_counts()/len(X_test)
intersection = site_domain_test.index.intersection(site_domain_train.index)

In [ ]:
def missing_replace(x):
    if x not in intersection:
        return "missing"
    else:
        return x

In [ ]:
X_train.site_domain = X_train.site_domain.apply(missing_replace)
X_test.site_domain = X_test.site_domain.apply(missing_replace)

## site_category

In [ ]:
site_category_train = X_train.site_category.value_counts()/len(X_train)
site_category_test = X_test.site_category.value_counts()/len(X_test)
intersection = site_category_test.index.intersection(site_category_train.index)

In [ ]:
def missing_replace(x):
    if x not in intersection:
        return "missing"
    else:
        return x

In [ ]:
X_train.site_category = X_train.site_category.apply(missing_replace)
X_test.site_category = X_test.site_category.apply(missing_replace)

## app_id

In [ ]:
app_id_train = X_train.app_id.value_counts()/len(X_train)
app_id_test = X_test.app_id.value_counts()/len(X_test)
intersection = app_id_test.index.intersection(app_id_train.index)

In [ ]:
def missing_replace(x):
    if x not in intersection:
        return "missing"
    else:
        return x

In [ ]:
X_train.app_id = X_train.app_id.apply(missing_replace)
X_test.app_id = X_test.app_id.apply(missing_replace)

## app_domain

In [ ]:
app_domain_train = X_train.app_domain.value_counts()/len(X_train)
app_domain_test = X_test.app_domain.value_counts()/len(X_test)
intersection = app_domain_test.index.intersection(app_domain_train.index)

In [ ]:
def missing_replace(x):
    if x not in intersection:
        return "missing"
    else:
        return x

In [ ]:
X_train.app_domain = X_train.app_domain.apply(missing_replace)
X_test.app_domain = X_test.app_domain.apply(missing_replace)

## app_category

In [ ]:
app_category_train = X_train.app_category.value_counts()/len(X_train)
app_category_test = X_test.app_category.value_counts()/len(X_test)
intersection = app_category_test.index.intersection(app_category_train.index)

In [ ]:
def missing_replace(x):
    if x not in intersection:
        return "missing"
    else:
        return x

In [ ]:
X_train.app_category = X_train.app_category.apply(missing_replace)
X_test.app_category = X_test.app_category.apply(missing_replace)

## device_id

In [ ]:
device_id_train = X_train.device_id.value_counts()/len(X_train)
device_id_test = X_test.device_id.value_counts()/len(X_test)
intersection = device_id_test.index.intersection(device_id_train.index)

In [ ]:
def missing_replace(x):
    if x not in intersection:
        return "missing"
    else:
        return x

In [ ]:
X_train.device_id = X_train.device_id.apply(missing_replace)
X_test.device_id = X_test.device_id.apply(missing_replace)

## device_ip

In [ ]:
device_ip_train = X_train.device_ip.value_counts()/len(X_train)
device_ip_test = X_test.device_ip.value_counts()/len(X_test)
intersection = device_ip_test.index.intersection(device_ip_train.index)

In [ ]:
def missing_replace(x):
    if x not in intersection:
        return "missing"
    else:
        return x

In [ ]:
X_train.device_ip = X_train.device_ip.apply(missing_replace)
X_test.device_ip = X_test.device_ip.apply(missing_replace)

## device_model

In [ ]:
device_model_train = X_train.device_model.value_counts()/len(X_train)
device_model_test = X_test.device_model.value_counts()/len(X_test)
intersection = device_model_test.index.intersection(device_model_train.index)

In [ ]:
def missing_replace(x):
    if x not in intersection:
        return "missing"
    else:
        return x

In [ ]:
X_train.device_model = X_train.device_model.apply(missing_replace)
X_test.device_model = X_test.device_model.apply(missing_replace)

In [ ]:
X_train.to_pickle('data/train_xgb_miss')
X_test.to_pickle('data/test_xgb_miss')

## hash! 

In [ ]:
import pandas as pd 
X_train = pd.read_pickle('data/train_xgb_miss')
X_test = pd.read_pickle('data/test_xgb_miss')

In [ ]:
import hashlib

# try a md5 hash and apply it to string
def string2numeric_hash(text):
    if text=='missing':
        return 0 # xgboost treat 0 as missing
    return int(hashlib.md5(text.encode('utf-8')).hexdigest()[:8], 16)
 
# test
print (string2numeric_hash("this is a nice string"))
print (string2numeric_hash("loser"))
print (string2numeric_hash("missing"))

In [ ]:
X_train['site_id_md5']=X_train.site_id.apply(string2numeric_hash)
X_train['site_domain_md5']=X_train.site_domain.apply(string2numeric_hash)
X_train['site_category_md5']=X_train.site_category.apply(string2numeric_hash)

X_train['app_id_md5']=X_train.app_id.apply(string2numeric_hash)
X_train['app_domain_md5']=X_train.app_domain.apply(string2numeric_hash)
X_train['app_category_md5']=X_train.app_category.apply(string2numeric_hash)

X_train['device_id_md5']=X_train.device_id.apply(string2numeric_hash)
X_train['device_ip_md5']=X_train.device_ip.apply(string2numeric_hash)
X_train['device_model_md5']=X_train.device_model.apply(string2numeric_hash)

In [ ]:
X_train['weekday']= (X_train['hour']%10000/100).astype(int)%7
X_train['hour_']= (X_train['hour']%100)

In [ ]:
X_train_features = X_train[['site_id_md5','site_domain_md5','site_category_md5',\
                        'app_id_md5','app_domain_md5','app_category_md5',\
                        'device_id_md5','device_ip_md5','device_model_md5',\
                        'weekday','hour_','banner_pos','C1','device_type','device_conn_type',\
                        'C14','C15','C16','C17','C18','C19','C20','C21']].values

In [ ]:
labels = X_train.click.values

In [ ]:
import numpy as np
np.save('train_xgb_hash',X_train_features)
np.save('train_xgb_y',labels)

## Hash on Testing

In [ ]:
X_test['site_id_md5'] = X_test.site_id.apply(string2numeric_hash)
X_test['site_domain_md5'] = X_test.site_domain.apply(string2numeric_hash)
X_test['site_category_md5'] = X_test.site_category.apply(string2numeric_hash)

X_test['app_id_md5'] = X_test.app_id.apply(string2numeric_hash)
X_test['app_domain_md5'] = X_test.app_domain.apply(string2numeric_hash)
X_test['app_category_md5'] = X_test.app_category.apply(string2numeric_hash)

X_test['device_id_md5'] = X_test.device_id.apply(string2numeric_hash)
X_test['device_ip_md5'] = X_test.device_ip.apply(string2numeric_hash)
X_test['device_model_md5'] = X_test.device_model.apply(string2numeric_hash)

In [ ]:
X_test['weekday']= (X_test['hour']%10000/100).astype(int)%7
X_test['hour_']= (X_test['hour']%100)

In [ ]:
X_test_features = X_test[['site_id_md5','site_domain_md5','site_category_md5',\
                        'app_id_md5','app_domain_md5','app_category_md5',\
                        'device_id_md5','device_ip_md5','device_model_md5',\
                        'weekday','hour_','banner_pos','C1','device_type','device_conn_type',\
                        'C14','C15','C16','C17','C18','C19','C20','C21']].values

In [ ]:
import numpy as np
np.save('test_xgb_hash',X_test_features)

# [Sec3] Strart training XGB/lightgbm


In [ ]:
import numpy as np
X_train_features = np.load('train_xgb_hash.npy')
labels = np.load('train_xgb_y.npy')

## [Sec3-1] LightGBM
parameter relative to regularization of  lightgbm:
```
max_bin
min_data_in_leaf
min_sum_hessian_in_leaf
feature_fraction
bagging_fraction
bagging_freq
lambda_l1
lambda_l2
min_gain_to_split
num_leaves
```

[[ref]](https://github.com/Microsoft/LightGBM/blob/master/docs/Parameters.md)

In [ ]:
import lightgbm as lgb
#create lightgbm dataset
#msk = np.load("20170727_msk.npy")
msk = np.random.rand(len(X_train_features)) < 0.8

lgb_train = lgb.Dataset(X_train_features[msk], labels[msk])
lgb_eval = lgb.Dataset(X_train_features[~msk], labels[~msk],reference=lgb_train)


In [27]:
# specify your configurations as a dict
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': { 'binary_logloss'},
    'num_leaves':15 , # defauly leaves(31) amount for each tree
    'lambda_l2': 1,
    'learning_rate': 0.005,
    'feature_fraction': 0.5, # will select 50% features before training each tree
    'bagging_fraction': 0.3, #feature_fraction, but this will random select part of data
    'bagging_freq': 5, #  perform bagging at every 5 iteration
    'verbose': 0
}

print('Start training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=80000,
                valid_sets=lgb_eval,
                early_stopping_rounds=500)

[62872]	valid_0's binary_logloss: 0.393422
[62873]	valid_0's binary_logloss: 0.393422
[62874]	valid_0's binary_logloss: 0.393422
[62875]	valid_0's binary_logloss: 0.393422
[62876]	valid_0's binary_logloss: 0.393422
[62877]	valid_0's binary_logloss: 0.393421
[62878]	valid_0's binary_logloss: 0.393421
[62879]	valid_0's binary_logloss: 0.393421
[62880]	valid_0's binary_logloss: 0.393421
[62881]	valid_0's binary_logloss: 0.393421
[62882]	valid_0's binary_logloss: 0.393421
[62883]	valid_0's binary_logloss: 0.393421
[62884]	valid_0's binary_logloss: 0.393421
[62885]	valid_0's binary_logloss: 0.393421
[62886]	valid_0's binary_logloss: 0.393421
[62887]	valid_0's binary_logloss: 0.393421
[62888]	valid_0's binary_logloss: 0.393421
[62889]	valid_0's binary_logloss: 0.393421
[62890]	valid_0's binary_logloss: 0.393421
[62891]	valid_0's binary_logloss: 0.393421
[62892]	valid_0's binary_logloss: 0.393421
[62893]	valid_0's binary_logloss: 0.393421
[62894]	valid_0's binary_logloss: 0.393421
[62895]	val

[63064]	valid_0's binary_logloss: 0.393413
[63065]	valid_0's binary_logloss: 0.393413
[63066]	valid_0's binary_logloss: 0.393413
[63067]	valid_0's binary_logloss: 0.393413
[63068]	valid_0's binary_logloss: 0.393413
[63069]	valid_0's binary_logloss: 0.393413
[63070]	valid_0's binary_logloss: 0.393413
[63071]	valid_0's binary_logloss: 0.393413
[63072]	valid_0's binary_logloss: 0.393413
[63073]	valid_0's binary_logloss: 0.393413
[63074]	valid_0's binary_logloss: 0.393413
[63075]	valid_0's binary_logloss: 0.393413
[63076]	valid_0's binary_logloss: 0.393413
[63077]	valid_0's binary_logloss: 0.393413
[63078]	valid_0's binary_logloss: 0.393413
[63079]	valid_0's binary_logloss: 0.393413
[63080]	valid_0's binary_logloss: 0.393413
[63081]	valid_0's binary_logloss: 0.393413
[63082]	valid_0's binary_logloss: 0.393413
[63083]	valid_0's binary_logloss: 0.393413
[63084]	valid_0's binary_logloss: 0.393413
[63085]	valid_0's binary_logloss: 0.393413
[63086]	valid_0's binary_logloss: 0.393412
[63087]	val

[63256]	valid_0's binary_logloss: 0.393404
[63257]	valid_0's binary_logloss: 0.393404
[63258]	valid_0's binary_logloss: 0.393404
[63259]	valid_0's binary_logloss: 0.393404
[63260]	valid_0's binary_logloss: 0.393404
[63261]	valid_0's binary_logloss: 0.393404
[63262]	valid_0's binary_logloss: 0.393404
[63263]	valid_0's binary_logloss: 0.393404
[63264]	valid_0's binary_logloss: 0.393404
[63265]	valid_0's binary_logloss: 0.393404
[63266]	valid_0's binary_logloss: 0.393404
[63267]	valid_0's binary_logloss: 0.393404
[63268]	valid_0's binary_logloss: 0.393404
[63269]	valid_0's binary_logloss: 0.393404
[63270]	valid_0's binary_logloss: 0.393404
[63271]	valid_0's binary_logloss: 0.393404
[63272]	valid_0's binary_logloss: 0.393404
[63273]	valid_0's binary_logloss: 0.393404
[63274]	valid_0's binary_logloss: 0.393403
[63275]	valid_0's binary_logloss: 0.393403
[63276]	valid_0's binary_logloss: 0.393403
[63277]	valid_0's binary_logloss: 0.393403
[63278]	valid_0's binary_logloss: 0.393403
[63279]	val

[63448]	valid_0's binary_logloss: 0.393395
[63449]	valid_0's binary_logloss: 0.393395
[63450]	valid_0's binary_logloss: 0.393395
[63451]	valid_0's binary_logloss: 0.393395
[63452]	valid_0's binary_logloss: 0.393395
[63453]	valid_0's binary_logloss: 0.393395
[63454]	valid_0's binary_logloss: 0.393395
[63455]	valid_0's binary_logloss: 0.393395
[63456]	valid_0's binary_logloss: 0.393395
[63457]	valid_0's binary_logloss: 0.393395
[63458]	valid_0's binary_logloss: 0.393395
[63459]	valid_0's binary_logloss: 0.393395
[63460]	valid_0's binary_logloss: 0.393395
[63461]	valid_0's binary_logloss: 0.393394
[63462]	valid_0's binary_logloss: 0.393394
[63463]	valid_0's binary_logloss: 0.393394
[63464]	valid_0's binary_logloss: 0.393394
[63465]	valid_0's binary_logloss: 0.393394
[63466]	valid_0's binary_logloss: 0.393394
[63467]	valid_0's binary_logloss: 0.393394
[63468]	valid_0's binary_logloss: 0.393394
[63469]	valid_0's binary_logloss: 0.393394
[63470]	valid_0's binary_logloss: 0.393394
[63471]	val

[63639]	valid_0's binary_logloss: 0.393386
[63640]	valid_0's binary_logloss: 0.393386
[63641]	valid_0's binary_logloss: 0.393385
[63642]	valid_0's binary_logloss: 0.393385
[63643]	valid_0's binary_logloss: 0.393385
[63644]	valid_0's binary_logloss: 0.393385
[63645]	valid_0's binary_logloss: 0.393385
[63646]	valid_0's binary_logloss: 0.393385
[63647]	valid_0's binary_logloss: 0.393385
[63648]	valid_0's binary_logloss: 0.393385
[63649]	valid_0's binary_logloss: 0.393385
[63650]	valid_0's binary_logloss: 0.393385
[63651]	valid_0's binary_logloss: 0.393385
[63652]	valid_0's binary_logloss: 0.393385
[63653]	valid_0's binary_logloss: 0.393385
[63654]	valid_0's binary_logloss: 0.393385
[63655]	valid_0's binary_logloss: 0.393385
[63656]	valid_0's binary_logloss: 0.393385
[63657]	valid_0's binary_logloss: 0.393385
[63658]	valid_0's binary_logloss: 0.393385
[63659]	valid_0's binary_logloss: 0.393385
[63660]	valid_0's binary_logloss: 0.393385
[63661]	valid_0's binary_logloss: 0.393385
[63662]	val

...

[79154]	valid_0's binary_logloss: 0.392784
[79155]	valid_0's binary_logloss: 0.392784
[79156]	valid_0's binary_logloss: 0.392784
[79157]	valid_0's binary_logloss: 0.392784
[79158]	valid_0's binary_logloss: 0.392784
[79159]	valid_0's binary_logloss: 0.392784
[79160]	valid_0's binary_logloss: 0.392784
[79161]	valid_0's binary_logloss: 0.392784
[79162]	valid_0's binary_logloss: 0.392784
[79163]	valid_0's binary_logloss: 0.392784
[79164]	valid_0's binary_logloss: 0.392783
[79165]	valid_0's binary_logloss: 0.392783
[79166]	valid_0's binary_logloss: 0.392783
[79167]	valid_0's binary_logloss: 0.392783
[79168]	valid_0's binary_logloss: 0.392783
[79169]	valid_0's binary_logloss: 0.392783
[79170]	valid_0's binary_logloss: 0.392783
[79171]	valid_0's binary_logloss: 0.392783
[79172]	valid_0's binary_logloss: 0.392783
[79173]	valid_0's binary_logloss: 0.392783
[79174]	valid_0's binary_logloss: 0.392783
[79175]	valid_0's binary_logloss: 0.392783
[79176]	valid_0's binary_logloss: 0.392783
[79177]	val

[79346]	valid_0's binary_logloss: 0.392778
[79347]	valid_0's binary_logloss: 0.392777
[79348]	valid_0's binary_logloss: 0.392777
[79349]	valid_0's binary_logloss: 0.392777
[79350]	valid_0's binary_logloss: 0.392777
[79351]	valid_0's binary_logloss: 0.392777
[79352]	valid_0's binary_logloss: 0.392777
[79353]	valid_0's binary_logloss: 0.392777
[79354]	valid_0's binary_logloss: 0.392777
[79355]	valid_0's binary_logloss: 0.392777
[79356]	valid_0's binary_logloss: 0.392777
[79357]	valid_0's binary_logloss: 0.392777
[79358]	valid_0's binary_logloss: 0.392777
[79359]	valid_0's binary_logloss: 0.392777
[79360]	valid_0's binary_logloss: 0.392777
[79361]	valid_0's binary_logloss: 0.392777
[79362]	valid_0's binary_logloss: 0.392777
[79363]	valid_0's binary_logloss: 0.392777
[79364]	valid_0's binary_logloss: 0.392777
[79365]	valid_0's binary_logloss: 0.392777
[79366]	valid_0's binary_logloss: 0.392777
[79367]	valid_0's binary_logloss: 0.392777
[79368]	valid_0's binary_logloss: 0.392777
[79369]	val

[79537]	valid_0's binary_logloss: 0.392771
[79538]	valid_0's binary_logloss: 0.392771
[79539]	valid_0's binary_logloss: 0.392771
[79540]	valid_0's binary_logloss: 0.392771
[79541]	valid_0's binary_logloss: 0.392771
[79542]	valid_0's binary_logloss: 0.392771
[79543]	valid_0's binary_logloss: 0.392771
[79544]	valid_0's binary_logloss: 0.392771
[79545]	valid_0's binary_logloss: 0.39277
[79546]	valid_0's binary_logloss: 0.39277
[79547]	valid_0's binary_logloss: 0.39277
[79548]	valid_0's binary_logloss: 0.39277
[79549]	valid_0's binary_logloss: 0.39277
[79550]	valid_0's binary_logloss: 0.39277
[79551]	valid_0's binary_logloss: 0.39277
[79552]	valid_0's binary_logloss: 0.39277
[79553]	valid_0's binary_logloss: 0.39277
[79554]	valid_0's binary_logloss: 0.39277
[79555]	valid_0's binary_logloss: 0.39277
[79556]	valid_0's binary_logloss: 0.39277
[79557]	valid_0's binary_logloss: 0.39277
[79558]	valid_0's binary_logloss: 0.39277
[79559]	valid_0's binary_logloss: 0.39277
[79560]	valid_0's binary_l

[79729]	valid_0's binary_logloss: 0.392764
[79730]	valid_0's binary_logloss: 0.392764
[79731]	valid_0's binary_logloss: 0.392764
[79732]	valid_0's binary_logloss: 0.392764
[79733]	valid_0's binary_logloss: 0.392764
[79734]	valid_0's binary_logloss: 0.392764
[79735]	valid_0's binary_logloss: 0.392764
[79736]	valid_0's binary_logloss: 0.392764
[79737]	valid_0's binary_logloss: 0.392764
[79738]	valid_0's binary_logloss: 0.392764
[79739]	valid_0's binary_logloss: 0.392764
[79740]	valid_0's binary_logloss: 0.392763
[79741]	valid_0's binary_logloss: 0.392763
[79742]	valid_0's binary_logloss: 0.392763
[79743]	valid_0's binary_logloss: 0.392763
[79744]	valid_0's binary_logloss: 0.392763
[79745]	valid_0's binary_logloss: 0.392763
[79746]	valid_0's binary_logloss: 0.392763
[79747]	valid_0's binary_logloss: 0.392763
[79748]	valid_0's binary_logloss: 0.392763
[79749]	valid_0's binary_logloss: 0.392763
[79750]	valid_0's binary_logloss: 0.392763
[79751]	valid_0's binary_logloss: 0.392763
[79752]	val

[79921]	valid_0's binary_logloss: 0.392758
[79922]	valid_0's binary_logloss: 0.392758
[79923]	valid_0's binary_logloss: 0.392758
[79924]	valid_0's binary_logloss: 0.392758
[79925]	valid_0's binary_logloss: 0.392758
[79926]	valid_0's binary_logloss: 0.392758
[79927]	valid_0's binary_logloss: 0.392758
[79928]	valid_0's binary_logloss: 0.392758
[79929]	valid_0's binary_logloss: 0.392758
[79930]	valid_0's binary_logloss: 0.392758
[79931]	valid_0's binary_logloss: 0.392757
[79932]	valid_0's binary_logloss: 0.392757
[79933]	valid_0's binary_logloss: 0.392757
[79934]	valid_0's binary_logloss: 0.392757
[79935]	valid_0's binary_logloss: 0.392757
[79936]	valid_0's binary_logloss: 0.392757
[79937]	valid_0's binary_logloss: 0.392757
[79938]	valid_0's binary_logloss: 0.392757
[79939]	valid_0's binary_logloss: 0.392757
[79940]	valid_0's binary_logloss: 0.392757
[79941]	valid_0's binary_logloss: 0.392757
[79942]	valid_0's binary_logloss: 0.392757
[79943]	valid_0's binary_logloss: 0.392757
[79944]	val

In [ ]:
gbm.save_model('lightgbm_20170831.txt')

In [ ]:
import lightgbm as lgb
import numpy as np
#restore model
gbm = lgb.Booster(model_file='lightgbm_20170831.txt')


In [ ]:
X_test_features = np.load('test_xgb_hash.npy')

In [ ]:
y_hat = gbm.predict(X_test_features,num_iteration=80000) 
import pandas as pd
sampleSubmission = pd.read_csv("data/sampleSubmission")
sampleSubmission.click = y_hat
# with a naive feature hashing (0.408, 1142/1604)
sampleSubmission.to_csv("lightgbm_20170831_80000.csv",index=False)

In [ ]:
y_hat = gbm.predict(X_test_features,num_iteration=60000) 
import pandas as pd
sampleSubmission = pd.read_csv("data/sampleSubmission")
sampleSubmission.click = y_hat
# with a naive feature hashing (0.408, 1142/1604)
sampleSubmission.to_csv("lightgbm_20170831_60000.csv",index=False)

In [25]:
y_hat = gbm.predict(X_test_features,num_iteration=20000) 
import pandas as pd
sampleSubmission = pd.read_csv("data/sampleSubmission")
sampleSubmission.click = y_hat
# with a naive feature hashing (0.408, 1142/1604)
sampleSubmission.to_csv("lightgbm_20170831_20000.csv",index=False)

In [26]:
y_hat = gbm.predict(X_test_features,num_iteration=10000) 
import pandas as pd
sampleSubmission = pd.read_csv("data/sampleSubmission")
sampleSubmission.click = y_hat
# with a naive feature hashing (0.408, 1142/1604)
sampleSubmission.to_csv("lightgbm_20170831_10000.csv",index=False)

## [Sec3-2] XGBoost

In [ ]:
import xgboost as xgb
param = {'max_depth':6, 'eta':0.01, 'silent':1, 
         'objective':'binary:logistic' ,'subsample':0.3,
        'min_child_weight':2,'lambda':2,'tree_method':'approx'}
param['nthread'] = 4
param['colsample_bytree'] = 0.5
param['eval_metric'] = 'logloss'

#msk = np.random.rand(len(X_train)) < 0.8
msk = np.load("20170727_msk.npy")

dtrain = xgb.DMatrix(X_train_features[msk], labels[msk])
dvali = xgb.DMatrix(X_train_features[~msk], labels[~msk])

In [ ]:
num_round = 30000
evallist  = [ (dtrain,'train'),(dvali,'eval')]
bst = xgb.train( param, dtrain, num_round, evallist , early_stopping_rounds=1000)

In [ ]:
bst.best_score

In [ ]:
bst.save_model('md_hash_0815.model')

## TEsting 

In [ ]:
import xgboost as xgb

In [ ]:
bst = xgb.Booster({'nthread':4}) #init model
bst.load_model("md_hash_0815.model") # load model
dtest = xgb.DMatrix(tmp_features)
ypred = bst.predict(dtest,ntree_limit=300)
sampleSubmission = pd.read_csv("data/sampleSubmission")
#sampleSubmission.click = (y_site+y_app)/2
sampleSubmission.click = ypred 
sampleSubmission.to_csv("md_hash_0815_300.csv",index=False)


In [ ]:
bst.get_fscore()

In [ ]:
%matplotlib inline

xgb.plot_importance(bst)